# **Academic Mobility Flows (1980-2022)**

# Global Talent Flows: The Academic Mobility Flows Webtool

**Author:** Marcia R. Ferreira (Complexity Science Hub Vienna & TU Wien)


*The best way to send information is to wrap it up in a person.*
    (Julius Robert Oppenheimer, 1948)


- How do academic talents travel around the world?
- Where do researchers go when they change affiliations? And where do they come from?
- Which research institutions, regions or countries are hotspots for scientific expertise in specific research areas?
- And which of these institutions are knowledge sinks?
- Are there sinks and hotspots that researchers lose or gain disproportionate among researchers?

Welcome to the [Complexity Science Hub Vienna](https://https://www.csh.ac.at/) “Global Talent Flows” analytics notebook created in collaboration with the [Dimensions.ai](https://www.dimensions.ai/), provides insights that go beyond traditional performance indicators. Academic mobility, as measured by changes in author affiliations, is at the heart of our investigation. Thus, demonstrating that bibliometric data can provide deep insights into policy, researcher mobility, and research entities attraction strategies, all of which are still understudied aspects of Quantitative Science Studies.
Data source: The data was provided by Dimensions.ai. We welcome feedback on our data visualization, and our scientific research. Explore the high-granularity mobility of researchers (aggregated) for thousands of publication-producing institutions across the globe.

**Disclaimer:** This data is not intended for commercialization or consulting and the data cannot be provided upon request!

For more information about the project contact me on here: [Márcia R. Ferreira](https://www.csh.ac.at/researcher/marcia-ferreira/)


- **Input:** Dimensions database on BigQuery
- **Output:**
Folder containing documentation about the project:
- https://drive.google.com/drive/folders/1Ac7nL2zzi8Q1crN1NaoSzzk_8YSBxD60?usp=sharing
- **Latest version:** https://vis.csh.ac.at/skill-flows/#
- **Github repository:** https://github.com/MarciaFG/skill-flow


## Initialization

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Sun Nov 26 17:07:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Drivers

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
#!pip install pynput
#pip install plotly==5.4.0
# libraries
import psutil
import humanize
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import torch
import nltk
import GPUtil as GPU
import plotly.graph_objs as go
import plotly.io as pio

# plotting
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from google.cloud import bigquery
from google.colab import files
%load_ext google.colab.data_table
%load_ext google.cloud.bigquery

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7395 sha256=4c0d5b831b1d0521ab459733a0525b562fd4ee86c0636527976e12382f4dd0be
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


**Loading data from Google Drive (If needed)**

In [ ]:
# run this to upload files
# from google.colab import files
# uploaded = files.upload()

**Mounting the Google Drive folder**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Runtime credentials**

In [4]:
# Provide your credentials to the runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Declare the Cloud project ID which will be used throughout this notebook**

In [6]:
# declare your project
project_id = "cshdimensionstest"

# START

# **PART I - Preprocessing**
## **1980-2022**
- Filtering by authors who have at least 2 publications
- Filtering by authors whose first publication year is at least 1980
- Filtering by authors who have published between 1980 and 2022

## **(1) Load Data**

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.intermediary_country AS

WITH researcher_first_pub_year AS (
  SELECT
    researcher_id,
    MIN(p.year) AS first_pub_year
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers
  GROUP BY
    researcher_id
  HAVING MIN(p.year) >= 1960
),
unnested_grid_ids AS (
SELECT
  researchers.researcher_id,
  grid as grid_ids,
  p.id AS pub_id,
  p.year,
  category.id
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(authors) AS researchers,
  UNNEST(researchers.grid_ids) AS grid,
  UNNEST(category_for.first_level.full) AS category
  JOIN
    researcher_first_pub_year rp
  ON
    rp.researcher_id = researchers.researcher_id
  WHERE
    category.id IS NOT NULL                     -- only publications that have a category id
    AND researchers.researcher_id IS NOT NULL
    AND researchers.grid_ids IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022            -- only publications between 1980-2022
)
SELECT DISTINCT a.*, b.country, b.country_code
FROM unnested_grid_ids a
JOIN cshdimensionstest.test.organisations_country b
ON a.grid_ids=b.id;

SELECT *
FROM cshdimensionstest.test.intermediary_country
ORDER BY researcher_id, year, pub_id, grid_ids
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,grid_ids,pub_id,year,id,country,country_code
0,ur.01000000010.53,grid.461843.c,pub.1077606951,2007,80003,China,CN
1,ur.01000000010.53,grid.461843.c,pub.1028748827,2011,80003,China,CN
2,ur.01000000010.53,grid.461843.c,pub.1040612737,2012,80003,China,CN
3,ur.01000000010.53,grid.461843.c,pub.1004493655,2013,80003,China,CN
4,ur.01000000010.53,grid.461843.c,pub.1039762887,2013,80003,China,CN
5,ur.01000000010.53,grid.461843.c,pub.1040126984,2013,80003,China,CN
6,ur.01000000010.53,grid.461843.c,pub.1049771699,2013,80003,China,CN
7,ur.01000000010.53,grid.461843.c,pub.1078832884,2013,80003,China,CN
8,ur.01000000010.53,grid.461843.c,pub.1121667807,2013,80003,China,CN
9,ur.01000000010.53,grid.461843.c,pub.1121815093,2013,80003,China,CN


In [7]:
%%bigquery --project $project_id
select count(distinct researcher_id) from cshdimensionstest.test.intermediary_country -- 19225774 same as the original intermediary table

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,19225774


In [9]:
%%bigquery --project $project_id
select count(distinct id) from cshdimensionstest.test.intermediary_country -- 22 for categories

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,22


###*(1.1) Population Statistics*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country AS

WITH total_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS total_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  GROUP BY
    country_code,
    country,
    year
),
researcher_grid_counts AS (
  SELECT
    researcher_id,
    COUNT(DISTINCT country_code) AS country_count
  FROM
    cshdimensionstest.test.intermediary_country
  GROUP BY
    researcher_id
),
non_mobile_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS non_mobile_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count = 1)
  GROUP BY
    country_code,
    country,
    year
),
mobile_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS mobile_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count > 1)
  GROUP BY
    country_code,
    country,
    year
)
SELECT
  tr.country_code,
  tr.country,
  tr.year,
  tr.total_researchers,
  COALESCE(nm.non_mobile_researchers, 0) AS non_mobile_researchers,
  COALESCE(mr.mobile_researchers, 0) AS mobile_researchers,
  ROUND((COALESCE(nm.non_mobile_researchers, 0) * 100.0) / tr.total_researchers, 2) AS pct_non_mobile,
  ROUND((COALESCE(mr.mobile_researchers, 0) * 100.0) / tr.total_researchers, 2) AS pct_mobile
FROM
  total_researchers tr
LEFT JOIN
  non_mobile_researchers nm
  ON tr.country_code = nm.country_code AND tr.year = nm.year
LEFT JOIN
  mobile_researchers mr
  ON tr.country_code = mr.country_code AND tr.year = mr.year
ORDER BY
  tr.country_code, tr.year;

SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country
ORDER BY country_code, year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,country,year,total_researchers,non_mobile_researchers,mobile_researchers,pct_non_mobile,pct_mobile
0,AD,Andorra,2009,1,1,0,100.00,0.00
1,AD,Andorra,2014,3,3,0,100.00,0.00
2,AD,Andorra,2015,1,1,0,100.00,0.00
3,AD,Andorra,2017,2,1,1,50.00,50.00
4,AD,Andorra,2018,1,1,0,100.00,0.00
5,AD,Andorra,2019,1,0,1,0.00,100.00
6,AD,Andorra,2020,4,2,2,50.00,50.00
7,AD,Andorra,2021,6,5,1,83.33,16.67
8,AD,Andorra,2022,5,3,2,60.00,40.00
9,AE,United Arab Emirates,1980,5,5,0,100.00,0.00


In [ ]:
%%bigquery --project $project_id
SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country
WHERE country_code = "US"
ORDER BY country_code, year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,country,year,total_researchers,non_mobile_researchers,mobile_researchers,pct_non_mobile,pct_mobile
0,US,United States,1980,107839,83562,24277,77.49,22.51
1,US,United States,1981,118548,90731,27817,76.54,23.46
2,US,United States,1982,123739,93655,30084,75.69,24.31
3,US,United States,1983,130065,97230,32835,74.75,25.25
4,US,United States,1984,138458,102390,36068,73.95,26.05
5,US,United States,1985,148128,109020,39108,73.60,26.40
6,US,United States,1986,158332,115440,42892,72.91,27.09
7,US,United States,1987,180426,129775,50651,71.93,28.07
8,US,United States,1988,208539,150002,58537,71.93,28.07
9,US,United States,1989,225282,161417,63865,71.65,28.35


**Selecting mobile researchers for further analysis**

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_country AS

WITH total_researchers AS (
  SELECT
    researcher_id,
    country_code,
    country,
    grid_ids,
    pub_id,
    id,
    year
  FROM
    cshdimensionstest.test.intermediary_country
),
researcher_grid_counts AS (
  SELECT
    researcher_id,
    COUNT(DISTINCT country_code) AS country_count
  FROM
    cshdimensionstest.test.intermediary_country
  GROUP BY
    researcher_id
)
  SELECT
    researcher_id,
    country_code,
    country,
    grid_ids,
    pub_id,
    id,
    year
  FROM
    total_researchers
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count > 1);

SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_country
ORDER BY researcher_id, year, pub_id, grid_ids, country_code
limit 1000

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,country_code,country,grid_ids,pub_id,id,year
0,ur.010000000201.99,TW,Taiwan,grid.260542.7,pub.1040782254,80002,2014
1,ur.010000000201.99,TW,Taiwan,grid.260542.7,pub.1040782254,80001,2014
2,ur.010000000201.99,TW,Taiwan,grid.412046.5,pub.1040782254,80002,2014
3,ur.010000000201.99,TW,Taiwan,grid.412046.5,pub.1040782254,80001,2014
4,ur.010000000201.99,TW,Taiwan,grid.453140.7,pub.1040782254,80001,2014
...,...,...,...,...,...,...,...
995,ur.01000000757.03,KR,South Korea,grid.414966.8,pub.1084405724,80003,2017
996,ur.01000000757.03,KR,South Korea,grid.411947.e,pub.1085288657,80003,2017
997,ur.01000000757.03,KR,South Korea,grid.411947.e,pub.1085861178,80003,2017
998,ur.01000000757.03,KR,South Korea,grid.414966.8,pub.1085861178,80003,2017


In [10]:
%%bigquery --project $project_id
SELECT COUNT(DISTINCT country_code) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; -- 217

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,217


### other to check
- SELECT COUNT(DISTINCT researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; -- 2,037,951
- SELECT COUNT(DISTINCT grid_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --56,221
- SELECT COUNT(DISTINCT pub_id) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --32,852,373
- SELECT COUNT(DISTINCT country_Code) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --217


## (2) Trajectory Sequence

1.   We opt for a simplified career trajectory
2.   We take into account the first and last year of publication at a given country







###*(2.1) Simplified*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.simple_sequence_1980_2022_country AS

SELECT DISTINCT
    researcher_id as researcher_ids,
    country_code,
    country,
    COUNT(DISTINCT pub_id) AS t_pubs,
    MIN(year) AS start_year,
    MAX(year) AS end_year,
    IFNULL(dense_rank() OVER (
        PARTITION BY researcher_id
        ORDER BY MIN(year), MAX(year) ASC
    ), 0) AS t
FROM cshdimensionstest.test.au_pub_history_1980_2022_country
GROUP BY researcher_id, country_code, country;

# Let's check the table
# ur.013012771111.87
# Rodrigo Costas

SELECT *
FROM cshdimensionstest.test.simple_sequence_1980_2022_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,country_code,country,t_pubs,start_year,end_year,t
0,ur.013012771111.87,ES,Spain,3,2007,2010,1
1,ur.013012771111.87,NL,Netherlands,82,2009,2022,2
2,ur.013012771111.87,ZA,South Africa,35,2017,2022,3


**ok we are good to go!**

# **PART II - Mobility Network**

## (3) Mobility Flows


### ***(3.1) Author Country Flows***
In this part we focus on flows at the level of countries

  **ATT: This is the most computationally expensive table, becareful with running it too many times**

In [ ]:
%%bigquery --project $project_id
# DROP TABLE IF EXISTS cshdimensionstest.test.simple_sequence_1980_2022_clustered_country;
CREATE TABLE cshdimensionstest.test.simple_sequence_1980_2022_clustered_country
(
  researcher_ids STRING,
  country_code STRING,
  start_year INT64,
  t INT64
)
CLUSTER BY researcher_ids
OPTIONS(
  description="A clustered table of simple_sequence_1980_2022"
)
AS
SELECT researcher_ids, country_code, start_year, t
FROM cshdimensionstest.test.simple_sequence_1980_2022_country;

# Now we have everything we need to construct the flows at the country level
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country AS
SELECT
    a.researcher_ids,
    a.country_code AS unit1,
    b.country_code AS unit2,
    a.start_year AS p1,
    b.start_year AS p2
FROM
    cshdimensionstest.test.simple_sequence_1980_2022_clustered_country a
    INNER JOIN cshdimensionstest.test.simple_sequence_1980_2022_clustered_country b
    ON a.researcher_ids = b.researcher_ids
WHERE a.t = b.t - 1;

# Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, p2, unit1, unit2
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2
0,ur.013012771111.87,ES,NL,2007,2009
1,ur.013012771111.87,NL,ZA,2009,2017


In [ ]:
# Check the table
# how many edges does it have?
%%bigquery --project $project_id
SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_country; -- 3,336,980 LINKS

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3336980


In [12]:
%%bigquery --project $project_id
SELECT * FROM cshdimensionstest.test.flows_1980_2022_country
WHERE unit1 = 'ES' and unit2='NL' and p1=p2
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2
0,ur.01247750327.98,ES,NL,1987,1987
1,ur.0670423527.32,ES,NL,1997,1997
2,ur.0713316325.69,ES,NL,1997,1997
3,ur.01345071625.40,ES,NL,1998,1998
4,ur.01366331056.52,ES,NL,1998,1998
5,ur.0727505426.83,ES,NL,2000,2000
6,ur.0613624405.83,ES,NL,2000,2000
7,ur.0654171775.32,ES,NL,2000,2000
8,ur.0607437367.53,ES,NL,2001,2001
9,ur.01153706730.95,ES,NL,2002,2002


 ### ***(3.2) Cross-Country Flows***

Flows exchanged between two countries at a given calendar year

In [ ]:
%%bigquery --project $project_id

-- Calculate the total flows between country pairs per year
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flows AS
SELECT
  unit1 AS geoid_o,
  unit2 AS geoid_d,
  p2 AS date_d,
  COUNT(DISTINCT researcher_ids) AS total_flows,
FROM cshdimensionstest.test.flows_1980_2022_country
GROUP BY
  geoid_o,
  geoid_d,
  date_d;

-- Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_flows
ORDER BY
  geoid_o,
  geoid_d,
  date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,geoid_o,geoid_d,date_d,total_flows
0,AE,AF,2018,1
1,AE,AF,2020,2
2,AE,AL,2019,1
3,AE,AL,2022,1
4,AE,AM,2018,1
5,AE,AM,2021,2
6,AE,AR,2012,1
7,AE,AT,2002,1
8,AE,AT,2003,1
9,AE,AT,2005,2


In [ ]:
%%bigquery --project $project_id
-- Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_flows
WHERE geoid_o = 'DE' AND geoid_d = 'US'
ORDER BY
  geoid_o,
  geoid_d,
  date_d
LIMIT 200;

Query is running:   0%|          |

Downloading:   0%|          |

,geoid_o,geoid_d,date_d,total_flows
0,DE,US,1980,203
1,DE,US,1981,239
2,DE,US,1982,289
3,DE,US,1983,277
4,DE,US,1984,278
5,DE,US,1985,274
6,DE,US,1986,357
7,DE,US,1987,471
8,DE,US,1988,476
9,DE,US,1989,573



### ***(3.3) Total Flows by Country***

***Overall flows by country over time***
- outgoing flows (counts and percentage)
- incoming flows (counts and percentage)
- total flows (counts)
- total net flows (counts)
- net_mobility_rate


In [ ]:
# @title not use
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_total_flows_agg AS
SELECT
  node,
  date_d,
  SUM(outgoing_flows) AS outgoing_flows,
  ROUND(IFNULL(SUM(outgoing_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 0), 1) AS percentage_outflows,
  SUM(incoming_flows) AS incoming_flows,
  ROUND(IFNULL(SUM(incoming_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 0), 1) AS percentage_inflows,
  SUM(outgoing_flows) + SUM(incoming_flows) AS total_flows,
  SUM(incoming_flows) - SUM(outgoing_flows) AS total_net_flows,
  ROUND(IFNULL((SUM(incoming_flows) - SUM(outgoing_flows)) / (SUM(incoming_flows) + SUM(outgoing_flows)) * 100, 0), 1) AS net_mobility_rate
FROM (
  SELECT
    unit1 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS outgoing_flows,
    0 AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit1, p2

  UNION ALL

  SELECT
    unit2 AS node,
    p2 as date_d,
    0 AS outgoing_flows,
    COUNT(DISTINCT researcher_ids) AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit2, p2
)
AS flows
GROUP BY
  node, date_d;

-- Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_total_flows_agg
ORDER BY
  node,
  date_d
LIMIT 50;

In [ ]:
%%bigquery --project $project_id


CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_total_flows_agg AS

WITH Outgoing AS (
  SELECT
    unit1 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS outgoing_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit1, p2
),
Incoming AS (
  SELECT
    unit2 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit2, p2
)
SELECT
  COALESCE(o.node, i.node) AS node,
  COALESCE(o.date_d, i.date_d) AS date_d,
  COALESCE(o.outgoing_flows, 0) AS outgoing_flows,
  COALESCE(i.incoming_flows, 0) AS incoming_flows,
  COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0) AS total_flows,
  COALESCE(i.incoming_flows, 0) - COALESCE(o.outgoing_flows, 0) AS total_net_flows,
  IF(COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0) > 0,
     ROUND(COALESCE(o.outgoing_flows, 0) / (COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0)) * 100, 1),
     NULL) AS percentage_outflows,
  IF(COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0) > 0,
     ROUND(COALESCE(i.incoming_flows, 0) / (COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0)) * 100, 1),
     NULL) AS percentage_inflows,
  IF(COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0) > 0,
     ROUND((COALESCE(i.incoming_flows, 0) - COALESCE(o.outgoing_flows, 0)) / (COALESCE(o.outgoing_flows, 0) + COALESCE(i.incoming_flows, 0)) * 100, 1),
     NULL) AS net_mobility_rate
FROM
  Outgoing o
FULL JOIN
  Incoming i
ON
  o.node = i.node AND o.date_d = i.date_d
ORDER BY
  node, date_d;

-- Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_total_flows_agg
ORDER BY
  node,
  date_d
LIMIT 50;


Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,incoming_flows,total_flows,total_net_flows,percentage_outflows,percentage_inflows,net_mobility_rate
0,AD,2017,0,1,1,1,0.0,100.0,100.0
1,AD,2020,0,1,1,1,0.0,100.0,100.0
2,AD,2022,0,1,1,1,0.0,100.0,100.0
3,AE,1982,0,1,1,1,0.0,100.0,100.0
4,AE,1983,0,1,1,1,0.0,100.0,100.0
5,AE,1984,0,1,1,1,0.0,100.0,100.0
6,AE,1985,1,2,3,1,33.3,66.7,33.3
7,AE,1986,0,6,6,6,0.0,100.0,100.0
8,AE,1987,1,3,4,2,25.0,75.0,50.0
9,AE,1988,2,5,7,3,28.6,71.4,42.9


In [ ]:
%%bigquery --project $project_id
-- Check the table
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_total_flows_agg
WHERE node = "AT"
ORDER BY
  node,
  date_d
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,incoming_flows,total_flows,total_net_flows,percentage_outflows,percentage_inflows,net_mobility_rate
0,AT,1980,47,65,112,18,42.0,58.0,16.1
1,AT,1981,88,54,142,-34,62.0,38.0,-23.9
2,AT,1982,93,107,200,14,46.5,53.5,7.0
3,AT,1983,83,72,155,-11,53.5,46.5,-7.1
4,AT,1984,112,112,224,0,50.0,50.0,0.0
5,AT,1985,104,109,213,5,48.8,51.2,2.3
6,AT,1986,113,98,211,-15,53.6,46.4,-7.1
7,AT,1987,205,324,529,119,38.8,61.2,22.5
8,AT,1988,212,158,370,-54,57.3,42.7,-14.6
9,AT,1989,228,237,465,9,49.0,51.0,1.9


## (4) Indicators

### ***(4.1) Academic Age***
The average academic age of inflowing and outflowing researchers for each node and year by joining the two subsets of data on the researcher ID. Note that the following only estimates the academic age at the time when the new affiliation enters the affiliation stream of the researcher. It does not reflect real movement.

In [24]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.researcher_first_pub_year AS
SELECT
  researchers.researcher_id,
  MIN(p.year) AS first_pub_year
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(p.authors) as researchers,
  UNNEST(researchers.grid_ids) as grid,
  UNNEST(p.category_for.first_level.full) as category
WHERE
category.id IS NOT NULL
  AND researchers.researcher_id IS NOT NULL
  AND researchers.grid_ids IS NOT NULL
GROUP BY
  researchers.researcher_id
HAVING
  MIN(p.year) >= 1960;


SELECT * FROM cshdimensionstest.test.researcher_first_pub_year LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,first_pub_year
0,ur.016152470121.86,1960
1,ur.015320127727.24,1960
2,ur.014347605516.22,1960
3,ur.0112412434.47,1960
4,ur.014171045116.00,1960
5,ur.011753634347.51,1960
6,ur.014622600431.97,1960
7,ur.011460155127.61,1960
8,ur.015702274037.20,1960
9,ur.011722052417.10,1960


In [25]:
%%bigquery --project $project_id
SELECT * FROM cshdimensionstest.test.researcher_first_pub_year WHERE researcher_id = 'ur.013012771111.87' LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,first_pub_year
0,ur.013012771111.87,2007


In [34]:
# @title NEW
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_country AS

WITH academic_age AS (
  SELECT
    a.researcher_id,
    b.unit1,
    b.unit2,
    a.first_pub_year,
    b.last_year_at_unit1,
    (b.last_year_at_unit1 - a.first_pub_year + 1) AS outgoing_academic_age,
    (b.last_year_at_unit2 - a.first_pub_year + 1) AS incoming_academic_age
  FROM cshdimensionstest.test.researcher_first_pub_year a
  JOIN (
    SELECT
      researcher_ids,
      unit1,
      unit2,
      MAX(p2) AS last_year_at_unit1,
      MAX(p2) AS last_year_at_unit2
    FROM cshdimensionstest.test.flows_1980_2022_country
    GROUP BY researcher_ids, unit1, unit2
  ) AS b
  ON a.researcher_id = b.researcher_ids
)
SELECT *
FROM academic_age;

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_1_country AS

SELECT a.*
  , IFNULL(b.outgoing_academic_age, 0) AS outgoing_academic_age
  , IFNULL(b.incoming_academic_age, 0) AS incoming_academic_age
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_age_country b
  ON a.researcher_ids=b.researcher_id
  AND a.unit1=b.unit1
  AND a.unit2=b.unit2;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_1_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, unit1, unit2
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age
0,ur.013012771111.87,ES,NL,2007,2009,3,3
1,ur.013012771111.87,NL,ZA,2009,2017,11,11


In [ ]:
# @title old not used
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_country AS

WITH academic_age AS (
  SELECT
    a.researcher_ids,
    b.unit1,
    b.unit2,
    a.min_academic_age,
    b.last_year_at_unit1,
    (b.last_year_at_unit1 - a.min_academic_age + 1) AS outgoing_academic_age,
    (b.last_year_at_unit2 - a.min_academic_age + 1) AS incoming_academic_age
  FROM (
    SELECT researcher_ids, MIN(p1) AS min_academic_age
    FROM cshdimensionstest.test.flows_1980_2022_country
    GROUP BY researcher_ids
  ) AS a
  JOIN (
    SELECT
      researcher_ids,
      unit1,
      unit2,
      MAX(p2) AS last_year_at_unit1,
      MAX(p2) AS last_year_at_unit2
    FROM cshdimensionstest.test.flows_1980_2022_country
    GROUP BY researcher_ids, unit1, unit2
  ) AS b
  ON a.researcher_ids = b.researcher_ids
)
SELECT *
FROM academic_age;

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_1_country AS

SELECT a.*
  , IFNULL(b.outgoing_academic_age, 0) AS outgoing_academic_age
  , IFNULL(b.incoming_academic_age, 0) AS incoming_academic_age
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_age_country b
  ON a.researcher_ids=b.researcher_ids
  AND a.unit1=b.unit1
  AND a.unit2=b.unit2;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_1_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, unit1, unit2
LIMIT 10;

#### *(4.1.1) Academic Age Groups*

In [36]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_deciles_country AS

WITH age_groups AS (
  SELECT
    researcher_ids,
    incoming_academic_age,
    CASE
      WHEN incoming_academic_age BETWEEN 1 AND 10 THEN 'Age 1-10'
      WHEN incoming_academic_age BETWEEN 11 AND 20 THEN 'Age 11-20'
      WHEN incoming_academic_age BETWEEN 21 AND 30 THEN 'Age 21-30'
      WHEN incoming_academic_age BETWEEN 31 AND 40 THEN 'Age 31-40'
      ELSE 'Age 41+'
    END AS age_group
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
)

SELECT
  a.*,
  b.age_group
FROM cshdimensionstest.test.flows_1980_2022_age_1_country a
JOIN age_groups b
  ON a.researcher_ids = b.researcher_ids
  AND a.incoming_academic_age = b.incoming_academic_age;

-- Check table:
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, p2, unit1, unit1
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age,age_group
0,ur.013012771111.87,ES,NL,2007,2009,3,3,Age 1-10
1,ur.013012771111.87,NL,ZA,2009,2017,11,11,Age 11-20


In [37]:
%%bigquery --project $project_id
SELECT COUNT(distinct researcher_ids) as n_researchers, age_group
FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
group by age_group
Order by age_group;

Query is running:   0%|          |

Downloading:   0%|          |

,n_researchers,age_group
0,1708397,Age 1-10
1,432922,Age 11-20
2,151625,Age 21-30
3,48858,Age 31-40
4,12128,Age 41+


In [7]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_groups_country AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d,
       COUNTIF(IFNULL(age_group, '') = 'Age 1-10') AS outgoing_age_1_10,
       COUNTIF(IFNULL(age_group, '') = 'Age 11-20') AS outgoing_age_11_20,
       COUNTIF(IFNULL(age_group, '') = 'Age 21-30') AS outgoing_age_21_30,
       COUNTIF(IFNULL(age_group, '') = 'Age 31-40') AS outgoing_age_31_40,
       COUNTIF(IFNULL(age_group, '') = 'Age 41+') AS outgoing_age_41_plus
FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
GROUP BY unit1, p2 )
,
incoming AS (
SELECT unit2 as node, p2 as date_d,
       COUNTIF(IFNULL(age_group, '') = 'Age 1-10') AS incoming_age_1_10,
       COUNTIF(IFNULL(age_group, '') = 'Age 11-20') AS incoming_age_11_20,
       COUNTIF(IFNULL(age_group, '') = 'Age 21-30') AS incoming_age_21_30,
       COUNTIF(IFNULL(age_group, '') = 'Age 31-40') AS incoming_age_31_40,
       COUNTIF(IFNULL(age_group, '') = 'Age 41+') AS incoming_age_41_plus
FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
GROUP BY unit2, date_d
)
, age_aggregated AS
(
SELECT a.*,
       b.incoming_age_1_10,
       b.incoming_age_11_20,
       b.incoming_age_21_30,
       b.incoming_age_31_40,
       b.incoming_age_41_plus
FROM outgoing a
LEFT JOIN incoming b
    ON a.node=b.node
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;


Query is running:   0%|          |

""


In [8]:
%%bigquery --project $project_id

-- Check table:
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_groups_country
ORDER BY node, date_d
LIMIT 200;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_age_1_10,outgoing_age_11_20,outgoing_age_21_30,outgoing_age_31_40,outgoing_age_41_plus,incoming_age_1_10,incoming_age_11_20,incoming_age_21_30,incoming_age_31_40,incoming_age_41_plus
0,AE,1985,0,1,0,0,0,2,0,0,0,0
1,AE,1987,1,0,0,0,0,2,1,0,0,0
2,AE,1988,2,0,0,0,0,3,2,0,0,0
3,AE,1989,0,1,0,0,0,5,1,2,0,0
4,AE,1990,2,0,0,0,0,3,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,AT,1981,79,23,2,0,0,44,19,1,0,0
196,AT,1982,77,33,1,0,0,91,30,2,0,0
197,AT,1983,73,19,1,0,0,59,24,6,0,0
198,AT,1984,98,31,2,0,0,79,50,2,0,0


#### *(4.1.2) Academic Age Statistics*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_statistics_country AS

WITH outgoing AS (
  SELECT
    unit1 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(outgoing_academic_age), 0) AS sum_outgoing_academic_age,
    ROUND(IFNULL(STDDEV(outgoing_academic_age), 0), 3) AS stdev_outgoing_academic_age,
    ROUND(IFNULL(APPROX_QUANTILES(outgoing_academic_age, 3)[OFFSET(1)], 0), 3) AS median_outgoing_academic_age,
    ROUND(IFNULL(SUM(outgoing_academic_age) / COUNT(DISTINCT researcher_ids), 0), 3) AS mean_outgoing_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
  GROUP BY unit1, p2
),
incoming AS (
  SELECT
    unit2 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(incoming_academic_age), 0) AS sum_incoming_academic_age,
    ROUND(IFNULL(STDDEV(incoming_academic_age), 0), 3) AS stdev_incoming_academic_age,
    ROUND(IFNULL(APPROX_QUANTILES(incoming_academic_age, 3)[OFFSET(1)], 0), 3) AS median_incoming_academic_age,
    ROUND(IFNULL(SUM(incoming_academic_age) / COUNT(DISTINCT researcher_ids), 0), 3) AS mean_incoming_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
  GROUP BY unit2, p2
),
age_aggregated AS (
  SELECT
    a.node,
    a.move_year,
    IFNULL(a.num_researchers, 0) as outflows,
    IFNULL(b.num_researchers, 0) as inflows,
    a.mean_outgoing_academic_age,
    IFNULL(stdev_outgoing_academic_age, 0) AS stdev_outgoing_academic_age,
    IFNULL(median_outgoing_academic_age, 0) AS median_outgoing_academic_age,
    b.mean_incoming_academic_age,
    IFNULL(stdev_incoming_academic_age, 0) AS stdev_incoming_academic_age,
    IFNULL(median_incoming_academic_age, 0) AS median_incoming_academic_age,
    CASE WHEN a.mean_outgoing_academic_age > 0 THEN ROUND(b.mean_incoming_academic_age / a.mean_outgoing_academic_age, 3) ELSE NULL END AS academic_age_ratio
  FROM outgoing a
  LEFT JOIN incoming b
  ON a.node = b.node AND a.move_year = b.move_year
)
SELECT *
FROM age_aggregated
ORDER BY node, move_year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_statistics_country
ORDER BY node, move_year
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,outflows,inflows,mean_outgoing_academic_age,stdev_outgoing_academic_age,median_outgoing_academic_age,mean_incoming_academic_age,stdev_incoming_academic_age,median_incoming_academic_age,academic_age_ratio
0,AE,1985,1,2,5.000,0.000,5.0,4.500,0.707,4.0,0.900
1,AE,1987,1,3,6.000,0.000,6.0,7.333,0.577,7.0,1.222
2,AE,1988,2,5,6.500,2.121,5.0,6.000,3.000,5.0,0.923
3,AE,1989,1,8,9.000,0.000,9.0,6.250,3.412,4.0,0.694
4,AE,1990,2,10,5.000,5.657,1.0,7.900,3.247,9.0,1.580
...,...,...,...,...,...,...,...,...,...,...,...
95,AM,1985,9,12,4.444,1.667,4.0,3.333,1.371,2.0,0.750
96,AM,1986,11,17,3.636,2.541,2.0,4.882,1.867,4.0,1.343
97,AM,1987,12,11,3.750,2.050,2.0,5.455,1.864,4.0,1.455
98,AM,1988,19,13,5.105,2.904,3.0,7.308,2.287,7.0,1.432


In [9]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flow_statistics AS

SELECT
  a.*,
  IFNULL(b.mean_outgoing_academic_age, 0) AS mean_outgoing_academic_age,
  IFNULL(b.stdev_outgoing_academic_age, 0) AS stdev_outgoing_academic_age,
  IFNULL(b.median_outgoing_academic_age, 0) AS median_outgoing_academic_age,
  IFNULL(b.mean_incoming_academic_age, 0) AS mean_incoming_academic_age,
  IFNULL(b.stdev_incoming_academic_age, 0) AS stdev_incoming_academic_age,
  IFNULL(b.median_incoming_academic_age, 0) AS median_incoming_academic_age
FROM
  cshdimensionstest.test.flows_1980_2022_country_total_flows_agg a
LEFT JOIN
  cshdimensionstest.test.flows_1980_2022_age_statistics_country b
ON
  a.node=b.node
  AND a.date_d=b.move_year;

-- Check table:
SELECT *
FROM
  cshdimensionstest.test.flows_1980_2022_country_flow_statistics
ORDER BY
  node,
  date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,incoming_flows,total_flows,total_net_flows,percentage_outflows,percentage_inflows,net_mobility_rate,mean_outgoing_academic_age,stdev_outgoing_academic_age,median_outgoing_academic_age,mean_incoming_academic_age,stdev_incoming_academic_age,median_incoming_academic_age
0,AD,2017,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
1,AD,2020,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
2,AD,2022,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
3,AE,1982,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
4,AE,1983,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
5,AE,1984,0,1,1,1,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
6,AE,1985,1,2,3,1,33.3,66.7,33.3,5.000,0.000,5.0,4.500,0.707,4.0
7,AE,1986,0,6,6,6,0.0,100.0,100.0,0.000,0.000,0.0,0.000,0.000,0.0
8,AE,1987,1,3,4,2,25.0,75.0,50.0,6.000,0.000,6.0,7.333,0.577,7.0
9,AE,1988,2,5,7,3,28.6,71.4,42.9,6.500,2.121,5.0,6.000,3.000,5.0


In [10]:
%%bigquery --project $project_id

-- Check table:
SELECT *
FROM
  cshdimensionstest.test.flows_1980_2022_country_flow_statistics
  WHERE node = 'AT'
ORDER BY
  node,
  date_d
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,incoming_flows,total_flows,total_net_flows,percentage_outflows,percentage_inflows,net_mobility_rate,mean_outgoing_academic_age,stdev_outgoing_academic_age,median_outgoing_academic_age,mean_incoming_academic_age,stdev_incoming_academic_age,median_incoming_academic_age
0,AT,1980,47,65,112,18,42.0,58.0,16.1,1.000,0.000,1.0,1.015,0.000,1.0
1,AT,1981,88,54,142,-34,62.0,38.0,-23.9,1.773,0.434,2.0,1.796,0.429,2.0
2,AT,1982,93,107,200,14,46.5,53.5,7.0,2.452,0.801,2.0,2.421,0.790,2.0
3,AT,1983,83,72,155,-11,53.5,46.5,-7.1,3.084,1.107,2.0,3.333,1.073,3.0
4,AT,1984,112,112,224,0,50.0,50.0,0.0,4.045,1.223,4.0,3.821,1.430,3.0
5,AT,1985,104,109,213,5,48.8,51.2,2.3,4.308,1.914,3.0,4.275,2.017,3.0
6,AT,1986,113,98,211,-15,53.6,46.4,-7.1,4.965,1.981,4.0,5.255,2.010,5.0
7,AT,1987,205,324,529,119,38.8,61.2,22.5,5.410,2.613,4.0,5.895,2.427,5.0
8,AT,1988,212,158,370,-54,57.3,42.7,-14.6,5.877,2.986,4.0,6.108,2.825,5.0
9,AT,1989,228,237,465,9,49.0,51.0,1.9,6.276,3.010,4.0,6.338,3.098,4.0


In [15]:
"""

import pandas as pd
countries = pd.read_csv('/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results/countries_metadata.csv', delimiter=',')
countries.head(10)
"""

In [ ]:
"""
!pip install pandas-gbq

import pandas_gbq

project_id = "cshdimensionstest"
dataset_id = "test"
table_id = "country_coordinates"

destination_table = f"{dataset_id}.{table_id}"

countries.to_gbq(destination_table, project_id, if_exists='replace')
"""

In [20]:
%%bigquery --project $project_id
select * from cshdimensionstest.test.country_coordinates

Query is running:   0%|          |

Downloading:   0%|          |

,row_no,country,latitude,longitude,name
0,0,GN,9.945587,-9.696645,Guinea
1,1,KI,-3.370417,-168.734039,Kiribati
2,2,LR,6.428055,-9.429499,Liberia
3,3,ME,42.708678,19.374390,Montenegro
4,4,PW,7.514980,134.582520,Palau
...,...,...,...,...,...
225,226,US,37.600000,-95.665000,United States
226,227,VA,41.903880,12.452083,Vatican
227,228,VC,13.252818,-61.197096,Saint Vincent and the Grenadines
228,229,VG,18.418089,-64.585431,Virgin Islands


#### *(4.1.3) Age and Distance Statistics*

In [ ]:
%%bigquery --project $project_id

# SOURCE DISTANCES
CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_source_country AS

WITH distance AS (
  SELECT
    a.researcher_ids,
    a.unit1 as source_country,
    a.unit2 as destination_country,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.country_coordinates b on a.unit1=b.country
LEFT JOIN cshdimensionstest.test.country_coordinates c on a.unit2=c.country
)
, distance_summary AS (
SELECT
  source_country
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as outflows
FROM distance
GROUP BY source_country, move_year )
SELECT
  source_country,
  move_year,
  average_distance_travelled_km,
  outflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY source_country, move_year;

# DESTINATION DISTANCES
CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_destination_country AS

WITH distance AS (
  SELECT
    a.researcher_ids,
    a.unit1 as source_country,
    a.unit2 as destination_country,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.country_coordinates b on a.unit1=b.country
LEFT JOIN cshdimensionstest.test.country_coordinates c on a.unit2=c.country
), distance_summary AS (
SELECT
  destination_country
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as inflows
FROM distance
GROUP BY destination_country, move_year )
SELECT
  destination_country,
  move_year,
  average_distance_travelled_km,
  inflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY destination_country, move_year;


CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances AS

SELECT
  a.*,
  IFNULL(b.average_distance_travelled_km, 0) AS mean_distance_travelled_to_destination,
  IFNULL(b.distance_category, '') AS distance_category_to_destination,
  IFNULL(c.average_distance_travelled_km, 0) AS mean_distance_travelled_from_source,
  IFNULL(c.distance_category, '') AS distance_category_from_source,
  IFNULL(d.total_researchers, 0) AS total_researchers,
  IFNULL(d.non_mobile_researchers, 0) AS non_mobile_researchers,
  IFNULL(d.pct_non_mobile, 0) AS pct_non_mobile,
  IFNULL(d.mobile_researchers, 0) AS mobile_researchers,
  IFNULL(d.pct_mobile, 0) AS pct_mobile
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics a
LEFT JOIN cshdimensionstest.test.average_distance_destination_country b
  ON a.node = b.destination_country AND a.date_d = b.move_year
LEFT JOIN cshdimensionstest.test.average_distance_source_country c
  ON a.node = c.source_country AND a.date_d = c.move_year
LEFT JOIN cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country d
 ON a.node = d.country_code and a.date_d =d.year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances
ORDER BY node, date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate,mean_outgoing_academic_age,...,incoming_academic_age_pct_change,mean_distance_travelled_to_destination,distance_category_to_destination,mean_distance_travelled_from_source,distance_category_from_source,total_researchers,non_mobile_researchers,pct_non_mobile,mobile_researchers,pct_mobile
0,AD,2017,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,2,1,50.00,1,50.00
1,AD,2020,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,502.025372,≥ 100 km < 1000 km,0.000000,,4,2,50.00,2,50.00
2,AD,2022,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,502.025372,≥ 100 km < 1000 km,0.000000,,4,2,50.00,2,50.00
3,AE,1986,0,0.0,2,100.0,2,2,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,8,2,25.00,6,75.00
4,AE,1987,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,10,5,50.00,5,50.00
5,AE,1988,1,25.0,3,75.0,4,2,50.0,5.000,...,0.000,0.000000,≥ 20000 km,0.000000,≥ 20000 km,18,11,61.11,7,38.89
6,AE,1989,0,0.0,4,100.0,4,4,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,19,9,47.37,10,52.63
7,AE,1990,1,33.3,2,66.7,3,1,33.3,1.000,...,-12.500,0.000000,≥ 20000 km,0.000000,≥ 20000 km,29,15,51.72,14,48.28
8,AE,1991,4,26.7,11,73.3,15,7,46.7,3.750,...,71.429,0.000000,≥ 20000 km,0.000000,≥ 20000 km,32,7,21.88,25,78.13
9,AE,1992,6,17.6,28,82.4,34,22,64.7,4.333,...,23.217,0.000000,≥ 20000 km,0.000000,≥ 20000 km,65,18,27.69,47,72.31


### ***(4.2) Top k Institutions***

#### *(4.2.1) Top Destination Institutions - not run*

In [ ]:
#@title old
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations AS
SELECT
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY origin_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
WHERE top_rank <= 5
GROUP BY
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
ORDER BY
  origin_node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations
where destination_node = 'grid.1001.0'
ORDER BY
  origin_node,
  date_d,
  top_rank ASC
LIMIT 1000;

#### *(4.2.2) Top Source Institutions - not run*

In [ ]:
#@title old
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources AS
SELECT
  destination_node,
  date_d,
  origin_node,
  top_rank,
  incoming_flows
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY destination_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
WHERE top_rank <= 5
GROUP BY
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
ORDER BY
  destination_node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources
where origin_node = 'grid.1001.0'
ORDER BY
  origin_node,
  date_d,
  top_rank ASC
LIMIT 1000;

#### *(4.2.2) Top Source/Destination Institutions*

In [21]:
#@title new
%%bigquery --project $project_id

# ALL DESTINATIONS

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_destinations AS

--Top 5 overall destination institutions for each source institution:
WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, p2 as date_d, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country, p2
),

destination_ranks AS (
  SELECT source_country, destination_country, date_d, count_80_22,
    ROW_NUMBER() OVER (PARTITION BY date_d, source_country ORDER BY  count_80_22 DESC) AS destination_rank
  FROM source_destination_counts
),

top_destinations_per_source AS (
  SELECT source_country, destination_country, date_d, count_80_22 AS destination_count_1980_2022, destination_rank AS destination_rank_1980_2022,
  FROM destination_ranks
--  WHERE destination_rank <= 5 AND count_80_22 > 1
)

SELECT
  source_country,
  destination_country,
  date_d,
  destination_count_1980_2022,
  destination_rank_1980_2022
FROM top_destinations_per_source
ORDER BY date_d, destination_rank_1980_2022, destination_count_1980_2022, source_country ASC;


# ALL SOURCES

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_sources AS

--Top 5 overall source institutions for each destination institution:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, p2 as date_d, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country, p2
),

source_ranks AS (
  SELECT source_country, destination_country, date_d, count_80_22,
    ROW_NUMBER() OVER (PARTITION BY  date_d, destination_country ORDER BY count_80_22 DESC) AS source_rank
  FROM source_destination_counts
),

top_sources_per_destination AS (
  SELECT source_country, destination_country, date_d, count_80_22 AS source_count_1980_2022, source_rank as source_rank_1980_2022
  FROM source_ranks
 -- WHERE source_rank <= 5 AND count_80_22 > 1
)

SELECT
  source_country,
  destination_country,
  date_d,
  source_count_1980_2022,
  source_rank_1980_2022
FROM top_sources_per_destination
ORDER BY date_d, source_rank_1980_2022, source_count_1980_2022, destination_country ASC;

Query is running:   0%|          |

""


In [24]:
%%bigquery --project $project_id
-- check tables
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_destinations
WHERE source_country = 'AT'
ORDER BY source_country, date_d, destination_rank_1980_2022
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,source_country,destination_country,date_d,destination_count_1980_2022,destination_rank_1980_2022
0,AT,US,1980,17,1
1,AT,DE,1980,12,2
2,AT,SE,1980,3,3
3,AT,CZ,1980,3,4
4,AT,GB,1980,3,5
...,...,...,...,...,...
95,AT,IE,1985,1,14
96,AT,RU,1985,1,15
97,AT,HU,1985,1,16
98,AT,TW,1985,1,17


In [23]:
%%bigquery --project $project_id
-- check tables
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_sources
ORDER BY destination_country, date_d, source_rank_1980_2022
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,source_country,destination_country,date_d,source_count_1980_2022,source_rank_1980_2022
0,US,AD,2017,1,1
1,ES,AD,2020,1,1
2,ES,AD,2022,1,1
3,IQ,AE,1982,1,1
4,EG,AE,1983,1,1
5,IQ,AE,1984,1,1
6,DE,AE,1985,1,1
7,EG,AE,1985,1,2
8,GB,AE,1986,3,1
9,CA,AE,1986,1,2


#### *(4.2.3) Top Overall Destination - Source Institutions*

In [25]:
%%bigquery --project $project_id

# TOP 5 DESTINATIONS

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_destinations_overall AS

--Top 5 overall destination countrys for each source country:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country
),

destination_ranks AS (
  SELECT source_country, destination_country, count_80_22,
    RANK() OVER (PARTITION BY source_country ORDER BY count_80_22 DESC) AS destination_rank
  FROM source_destination_counts
),

top_destinations_per_source AS (
  SELECT source_country, destination_country, count_80_22 AS destination_count_1980_2022, destination_rank AS destination_rank_1980_2022,
  FROM destination_ranks
--  WHERE destination_rank <= 5 AND count_80_22 > 1
)

SELECT
  source_country,
  destination_country,
  destination_count_1980_2022,
  destination_rank_1980_2022
FROM top_destinations_per_source
ORDER BY destination_rank_1980_2022, destination_count_1980_2022, source_country ASC;


# TOP 5 SOURCES

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_sources_overall AS

--Top 5 overall source countrys for each destination country:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country
),

source_ranks AS (
  SELECT source_country, destination_country, count_80_22,
    RANK() OVER (PARTITION BY destination_country ORDER BY count_80_22 DESC) AS source_rank
  FROM source_destination_counts
),

top_sources_per_destination AS (
  SELECT source_country, destination_country, count_80_22 AS source_count_1980_2022, source_rank as source_rank_1980_2022
  FROM source_ranks
 -- WHERE source_rank <= 5 AND count_80_22 > 1
)

SELECT
  source_country,
  destination_country,
  source_count_1980_2022,
  source_rank_1980_2022
FROM top_sources_per_destination
ORDER BY source_rank_1980_2022, source_count_1980_2022, destination_country ASC;

Query is running:   0%|          |

""


In [26]:
%%bigquery --project $project_id
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_destinations_overall
WHERE source_country = 'AT'
ORDER BY source_country, destination_rank_1980_2022
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,source_country,destination_country,destination_count_1980_2022,destination_rank_1980_2022
0,AT,DE,7051,1
1,AT,US,6272,2
2,AT,GB,3275,3
3,AT,CH,2517,4
4,AT,FR,1738,5
5,AT,IT,1738,5
6,AT,NL,1506,7
7,AT,CA,1325,8
8,AT,AU,1118,9
9,AT,ES,1090,10


In [27]:
%%bigquery --project $project_id
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_sources_overall
WHERE destination_country = 'AT'
ORDER BY destination_country, source_rank_1980_2022
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,source_country,destination_country,source_count_1980_2022,source_rank_1980_2022
0,DE,AT,8903,1
1,US,AT,7347,2
2,GB,AT,3418,3
3,IT,AT,2609,4
4,CH,AT,2274,5
5,FR,AT,2030,6
6,NL,AT,1624,7
7,ES,AT,1391,8
8,CA,AT,1360,9
9,SE,AT,1030,10


### ***(4.3) Productivity***

#### *(4.3.1) Author Productivity*

In [28]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_cumulative_pubs_1980_2022_country AS

WITH ordered_publications AS (
  SELECT DISTINCT
    researcher_id as researcher_ids,
    year,
    COUNT(DISTINCT pub_id) as n_pubs,
    ROW_NUMBER() OVER(PARTITION BY researcher_id ORDER BY year) AS publication_number
  FROM
    cshdimensionstest.test.au_pub_history_1980_2022_country
  GROUP BY researcher_id,  year
)
SELECT
  researcher_ids,
  year,
  SUM(n_pubs) OVER(PARTITION BY researcher_ids ORDER BY year
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                   ) AS cumulative_publications
FROM
  ordered_publications
ORDER BY
  researcher_ids,
  year;

SELECT *
FROM  cshdimensionstest.test.au_cumulative_pubs_1980_2022_country
WHERE researcher_ids = 'ur.01000000010.53'
ORDER BY year;

Query is running:   0%|          |

Downloading: |          |

,researcher_ids,year,cumulative_publications


In [29]:
%%bigquery --project $project_id
SELECT COUNT(DISTINCT researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022 -- 6,435,674

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,6435674


**Ok all good**

In [30]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_cum_pubs_country AS

SELECT
  f.researcher_ids,
  f.unit1,
  f.unit2,
  f.p1,
  f.p2,
  (
    SELECT
      MAX(a.cumulative_publications)
    FROM
      cshdimensionstest.test.au_cumulative_pubs_1980_2022_country a
    WHERE
       a.researcher_ids = f.researcher_ids
      AND a.year = f.p2
  ) AS source_cumulative_pubs
FROM
  cshdimensionstest.test.flows_1980_2022_country f;

SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country; --3,336,980

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3336980


In [31]:
%%bigquery --project $project_id
SELECT * FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country ORDER BY researcher_ids, p1, p2, unit1, unit2 LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,source_cumulative_pubs
0,ur.010000000201.99,TW,JP,2014,2020,4
1,ur.01000000021.08,JP,CA,2010,2013,3
2,ur.01000000021.08,CA,EG,2013,2016,4
3,ur.01000000021.08,EG,SA,2016,2021,9
4,ur.010000000667.71,PK,MX,2020,2022,2
5,ur.010000001435.52,IT,DE,2016,2021,9
6,ur.01000000145.32,US,CN,2001,2004,4
7,ur.01000000145.32,CN,GB,2004,2009,19
8,ur.01000000145.32,GB,SE,2009,2011,29
9,ur.01000000175.26,ES,DE,2010,2018,15


In [32]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_deciles_country AS
SELECT
  researcher_ids,
  unit1,
  unit2,
  p1,
  p2,
  source_cumulative_pubs,
  CASE
    WHEN source_cumulative_pubs BETWEEN 1 AND 3 THEN 'Productivity group id: 1 | Range (pubs): 1-3'
    WHEN source_cumulative_pubs BETWEEN 4 AND 10 THEN 'Productivity group id: 2 | Range (pubs): 4-10'
    WHEN source_cumulative_pubs BETWEEN 11 AND 25 THEN 'Productivity group id: 3 | Range (pubs): 11-25'
    WHEN source_cumulative_pubs BETWEEN 26 AND 50 THEN 'Productivity group id: 4 | Range (pubs): 26-50'
    WHEN source_cumulative_pubs BETWEEN 51 AND 70 THEN 'Productivity group id: 5 | Range (pubs): 51-70'
    WHEN source_cumulative_pubs BETWEEN 71 AND 100 THEN 'Productivity group id: 6 | Range (pubs): 71-100'
    WHEN source_cumulative_pubs BETWEEN 101 AND 250 THEN 'Productivity group id: 7 | Range (pubs): 101-250'
    WHEN source_cumulative_pubs BETWEEN 251 AND 500 THEN 'Productivity group id: 8 | Range (pubs): 251-500'
    WHEN source_cumulative_pubs BETWEEN 501 AND 1000 THEN 'Productivity group id: 9 | Range (pubs): 501-1000'
    ELSE 'Productivity group id: 10 | Range (pubs): 1001+' END AS pub_group
FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
GROUP BY
  researcher_ids,
  unit1,
  unit2,
  p1,
  p2,
  source_cumulative_pubs;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_deciles_country
ORDER BY researcher_ids, p1, p2, unit1, unit2
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,source_cumulative_pubs,pub_group
0,ur.010000000201.99,TW,JP,2014,2020,4,Productivity group id: 2 | Range (pubs): 4-10
1,ur.01000000021.08,JP,CA,2010,2013,3,Productivity group id: 1 | Range (pubs): 1-3
2,ur.01000000021.08,CA,EG,2013,2016,4,Productivity group id: 2 | Range (pubs): 4-10
3,ur.01000000021.08,EG,SA,2016,2021,9,Productivity group id: 2 | Range (pubs): 4-10
4,ur.010000000667.71,PK,MX,2020,2022,2,Productivity group id: 1 | Range (pubs): 1-3
5,ur.010000001435.52,IT,DE,2016,2021,9,Productivity group id: 2 | Range (pubs): 4-10
6,ur.01000000145.32,US,CN,2001,2004,4,Productivity group id: 2 | Range (pubs): 4-10
7,ur.01000000145.32,CN,GB,2004,2009,19,Productivity group id: 3 | Range (pubs): 11-25
8,ur.01000000145.32,GB,SE,2009,2011,29,Productivity group id: 4 | Range (pubs): 26-50
9,ur.01000000175.26,ES,DE,2010,2018,15,Productivity group id: 3 | Range (pubs): 11-25


In [33]:
%%bigquery --project $project_id
SELECT DISTINCT pub_group FROM  cshdimensionstest.test.flows_1980_2022_pub_deciles_country order by pub_group;

Query is running:   0%|          |

Downloading:   0%|          |

,pub_group
0,Productivity group id: 1 | Range (pubs): 1-3
1,Productivity group id: 10 | Range (pubs): 1001+
2,Productivity group id: 2 | Range (pubs): 4-10
3,Productivity group id: 3 | Range (pubs): 11-25
4,Productivity group id: 4 | Range (pubs): 26-50
5,Productivity group id: 5 | Range (pubs): 51-70
6,Productivity group id: 6 | Range (pubs): 71-100
7,Productivity group id: 7 | Range (pubs): 101-250
8,Productivity group id: 8 | Range (pubs): 251-500
9,Productivity group id: 9 | Range (pubs): 501-1000


#### *(4.3.2) Author Productivity Groups*

In [35]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_groups_country AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 1 | Range (pubs): 1-3') AS outgoing_pub_1_3_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 2 | Range (pubs): 4-10') AS outgoing_pub_4_10_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 3 | Range (pubs): 11-25') AS outgoing_pub_11_25_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 4 | Range (pubs): 26-50') AS outgoing_pub_26_50_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 5 | Range (pubs): 51-70') AS outgoing_pub_51_70_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 6 | Range (pubs): 71-100') AS outgoing_pub_71_100_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 7 | Range (pubs): 101-250') AS outgoing_pub_101_250_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 8 | Range (pubs): 251-500') AS outgoing_pub_251_500_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 9 | Range (pubs): 501-1000') AS outgoing_pub_501_1000_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 10 | Range (pubs): 1001+') AS outgoing_pub_1001_plus_y
     FROM  cshdimensionstest.test.flows_1980_2022_pub_deciles_country
GROUP BY unit1, p2 )
, incoming AS (
  SELECT unit2 as node, p2 as date_d,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 1 | Range (pubs): 1-3') AS incoming_pub_1_3_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 2 | Range (pubs): 4-10') AS incoming_pub_4_10_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 3 | Range (pubs): 11-25') AS incoming_pub_11_25_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 4 | Range (pubs): 26-50') AS incoming_pub_26_50_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 5 | Range (pubs): 51-70') AS incoming_pub_51_70_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 6 | Range (pubs): 71-100') AS incoming_pub_71_100_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 7 | Range (pubs): 101-250') AS incoming_pub_101_250_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 8 | Range (pubs): 251-500') AS incoming_pub_251_500_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 9 | Range (pubs): 501-1000') AS incoming_pub_501_1000_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 10 | Range (pubs): 1001+') AS incoming_pub_1001_plus_y
     FROM cshdimensionstest.test.flows_1980_2022_pub_deciles_country
GROUP BY unit2, date_d
) ,
age_aggregated AS
(
  SELECT a.*, b.incoming_pub_1_3_y, b.incoming_pub_4_10_y, b.incoming_pub_11_25_y,
  b.incoming_pub_26_50_y, b.incoming_pub_51_70_y, b.incoming_pub_71_100_y, b.incoming_pub_101_250_y,
  b.incoming_pub_251_500_y, b.incoming_pub_501_1000_y, b.incoming_pub_1001_plus_y
  FROM outgoing a
  LEFT JOIN incoming b
    ON a.node=b.node
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_groups_country
ORDER BY node, date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_pub_1_3_y,outgoing_pub_4_10_y,outgoing_pub_11_25_y,outgoing_pub_26_50_y,outgoing_pub_51_70_y,outgoing_pub_71_100_y,outgoing_pub_101_250_y,outgoing_pub_251_500_y,...,incoming_pub_1_3_y,incoming_pub_4_10_y,incoming_pub_11_25_y,incoming_pub_26_50_y,incoming_pub_51_70_y,incoming_pub_71_100_y,incoming_pub_101_250_y,incoming_pub_251_500_y,incoming_pub_501_1000_y,incoming_pub_1001_plus_y
0,AE,1985,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,AE,1987,0,1,0,0,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
2,AE,1988,0,0,1,1,0,0,0,0,...,3,1,1,0,0,0,0,0,0,0
3,AE,1989,0,1,0,0,0,0,0,0,...,4,3,0,1,0,0,0,0,0,0
4,AE,1990,2,0,0,0,0,0,0,0,...,1,2,4,2,1,0,0,0,0,0
5,AE,1991,2,2,0,0,0,0,0,0,...,5,7,4,2,1,0,0,0,0,0
6,AE,1992,5,2,0,0,0,0,0,0,...,8,15,6,6,0,2,0,0,0,0
7,AE,1993,2,1,1,0,0,0,0,0,...,5,8,6,2,0,0,0,0,0,0
8,AE,1994,4,1,3,2,0,0,0,0,...,5,11,12,7,2,1,0,0,0,0
9,AE,1995,3,3,2,3,0,1,1,0,...,3,10,9,2,0,0,0,0,0,0


#### *(4.3.3) Productivity Statistics*

In [36]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_stats_country AS

WITH
outgoing AS (
  SELECT
    unit1 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(source_cumulative_pubs), 0) AS sum_outgoing_pubs,
    IFNULL(STDDEV(source_cumulative_pubs), 0) AS stdev_outgoing_pubs,
    IFNULL(APPROX_QUANTILES(source_cumulative_pubs, 3)[OFFSET(1)], 0) AS median_outgoing_pubs
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
  GROUP BY unit1, p2
),
incoming AS (
  SELECT
    unit2 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(source_cumulative_pubs), 0) AS sum_incoming_pubs,
    IFNULL(STDDEV(source_cumulative_pubs), 0) AS stdev_incoming_pubs,
    IFNULL(APPROX_QUANTILES(source_cumulative_pubs, 3)[OFFSET(1)], 0) AS median_incoming_pubs
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
  GROUP BY unit2, p2
),
age_aggregated AS (
  SELECT
    a.node,
    a.move_year,
 --   a.num_researchers AS outflows,
 --   b.num_researchers AS inflows,
    ROUND(IFNULL(a.sum_outgoing_pubs / a.num_researchers, 0), 3) AS mean_outgoing_pubs,
    ROUND(IFNULL(a.sum_outgoing_pubs / (a.num_researchers * a.num_researchers), 0), 3) AS mean_normalized_outgoing_pubs,
    IFNULL(median_outgoing_pubs, 0) AS median_outgoing_pubs,
    IFNULL(sum_outgoing_pubs, 0) AS sum_outgoing_pubs,
    IFNULL(stdev_outgoing_pubs, 0) AS stdev_outgoing_pubs,
    ROUND(IFNULL(b.sum_incoming_pubs / b.num_researchers, 0), 3) AS mean_incoming_pubs,
    ROUND(IFNULL(b.sum_incoming_pubs / (b.num_researchers * b.num_researchers), 0), 3) AS mean_normalized_incoming_pubs,
    IFNULL(median_incoming_pubs, 0) AS median_incoming_pubs,
    IFNULL(sum_incoming_pubs, 0) AS sum_incoming_pubs,
    IFNULL(stdev_incoming_pubs, 0) AS stdev_incoming_pubs
FROM outgoing a
LEFT JOIN incoming b
  ON       a.node = b.node
  AND a.move_year = b.move_year
)
SELECT *
FROM age_aggregated
ORDER BY node, move_year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_stats_country
ORDER BY node, move_year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,mean_outgoing_pubs,mean_normalized_outgoing_pubs,median_outgoing_pubs,sum_outgoing_pubs,stdev_outgoing_pubs,mean_incoming_pubs,mean_normalized_incoming_pubs,median_incoming_pubs,sum_incoming_pubs,stdev_incoming_pubs
0,AE,1985,2.000,2.000,2,2,0.000000,3.000,1.500,2,6,1.414214
1,AE,1987,4.000,4.000,4,4,0.000000,7.667,2.556,5,23,3.055050
2,AE,1988,23.000,11.500,11,46,16.970563,6.600,1.320,2,33,6.767570
3,AE,1989,10.000,10.000,10,10,0.000000,7.000,0.875,2,56,9.546877
4,AE,1990,3.000,1.500,3,6,0.000000,19.700,1.970,11,197,17.448655
5,AE,1991,5.500,1.375,2,22,4.041452,13.579,0.715,5,258,15.896596
6,AE,1992,3.286,0.469,2,23,2.627691,16.811,0.454,5,622,21.277058
7,AE,1993,8.250,2.063,2,33,8.958236,10.048,0.478,4,211,8.440830
8,AE,1994,12.100,1.210,3,121,10.702544,20.000,0.556,8,720,19.274612
9,AE,1995,38.154,2.935,8,496,64.956198,12.917,0.538,8,310,9.650036


In [37]:
%%bigquery --project $project_id
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_stats_country
WHERE node = 'US'
ORDER BY move_year ASC;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,mean_outgoing_pubs,mean_normalized_outgoing_pubs,median_outgoing_pubs,sum_outgoing_pubs,stdev_outgoing_pubs,mean_incoming_pubs,mean_normalized_incoming_pubs,median_incoming_pubs,sum_incoming_pubs,stdev_incoming_pubs
0,US,1980,3.343,0.002,2,6834,2.729027,3.775,0.002,2,6856,3.341565
1,US,1981,4.659,0.002,2,11634,4.437904,4.480,0.002,2,10846,3.703674
2,US,1982,5.703,0.002,3,14019,5.791500,5.302,0.002,3,14549,4.708755
3,US,1983,6.700,0.003,3,17300,7.863561,6.144,0.002,3,17401,6.122368
4,US,1984,8.284,0.003,3,23942,9.032199,6.967,0.002,3,22107,7.033818
5,US,1985,9.321,0.003,3,26351,11.611508,7.564,0.002,3,25429,8.198683
6,US,1986,10.083,0.003,3,33396,12.813147,8.576,0.002,4,31783,9.770059
7,US,1987,11.296,0.003,3,46032,14.078272,9.302,0.002,4,47442,11.237338
8,US,1988,12.847,0.003,4,57004,16.908843,9.797,0.002,4,54059,12.179723
9,US,1989,14.237,0.003,4,68877,17.496054,10.468,0.002,4,60274,13.091643


This mean normalized calculation provides an indication of the overall productivity of the researchers that have joined the institution in a given calendar year. This indicator can help to assess the potential impact of their research on the group's output.

A value of 0.005 would mean that, on average, each researcher who has joined the group has had 0.005 publications before joining. Again, this value should be interpreted in the context of the specific group and field of research, but a low value could suggest that the group is not attracting highly productive or accomplished researchers.

Note that this indicator is highly affected by outliers.

In [38]:
# merging the indicators
%%bigquery --project $project_id
CREATE OR REPLACE TABLE  cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_production   AS
SELECT a.*,
    IFNULL(sum_outgoing_pubs, 0) AS total_outgoing_pubs,
    IFNULL(mean_outgoing_pubs, 0) AS mean_outgoing_pubs,
    IFNULL(mean_normalized_outgoing_pubs, 0) AS mean_normalized_outgoing_pubs,
    IFNULL(median_outgoing_pubs, 0) AS median_outgoing_pubs,
    IFNULL(stdev_outgoing_pubs, 0) AS stdev_outgoing_pubs,
    IFNULL(sum_incoming_pubs, 0) AS total_incoming_pubs,
    IFNULL(mean_incoming_pubs, 0) AS mean_incoming_pubs,
    IFNULL(mean_normalized_incoming_pubs, 0) AS mean_normalized_incoming_pubs,
    IFNULL(median_incoming_pubs, 0) AS median_incoming_pubs,
    IFNULL(stdev_incoming_pubs, 0) AS stdev_incoming_pubs
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_pub_stats_country b ON a.node=b.node AND a.date_d=b.move_year;

Query is running:   0%|          |

""


# END

#### *(4.4) Retention Rates*

To calculate the Scientists' retention rate per calendar year, I follow these steps:

    1. Identify the number of scientists affiliated in each institution in each year.
    2. Identify the number of scientists who were affiliated in the same institution in the previous year.
    3. Calculate the retention rate as the number of scientists who remained in the same institution divided by the total number of scientists enrolled in that institution.
    4. Repeat the process for each year and each institution.

This procedure calculates the retention rate of scientists affiliated with an institution from one year to the next. The retention rate is defined as *the ratio of the number of scientists affiliated with an institution in a given year to the number of scientists affiliated with the same institution in the previous year*.

The interpretation of the retention rate:

*A value of 1 means that the institution was able to retain the same number of scientists as the previous year, while a value greater than 1 indicates an increase in the number of affiliated scientists, and a value less than 1 indicates a decrease. The output can be used to assess the ability of institutions to retain their affiliated scientists over time*

This code calculates the retention rate of scientists for each institution and year, which is defined as the proportion of scientists who are affiliated with the institution in the current year and were also affiliated with the same institution in the previous year.

Advantages of this indicator include:

    It provides a clear picture of how well institutions are retaining their existing scientists.
    It is a simple indicator that can be easily understood.

Limitations of this indicator include:

    It does not take into account new scientists who may have joined the institution in the current year, so it may not provide a complete picture of the institution's ability to attract scientists.
    It assumes that the number of scientists affiliated with an institution in any given year is a good proxy for the institution's ability to attract and retain scientists. This may not always be the case, as factors such as funding, working conditions, and location can also play a significant role in attracting and retaining scientists.
    It also does not account for scientists who have left the institution but have not left the field entirely, which may lead to an over-estimation of the institution's retention rate.

The formula for the retention rate is calculated as:

retention_rate = affiliated_scientists_prev_year / affiliated_scientists

where:

    affiliated_scientists_prev_year represents the number of scientists affiliated with the institution in the previous year
    affiliated_scientists represents the number of scientists affiliated with the institution in the current year.

In [ ]:
#@title Retention test
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_retention AS
WITH scientist_retention
AS (
SELECT
research_orgs,
year,
researcher_ids,
LAG(research_orgs) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_research_orgs,
LAG(year) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_year,
FROM FROM cshdimensionstest.test.simple_sequence_1980_2022
WHERE year >= 1980 AND year <= 2022
),
retention_rate AS (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS remaining_scientists
FROM scientist_retention
WHERE research_orgs = prev_research_orgs AND year = prev_year + 2 -- checks how many scientists remained affiliated in the previous 2 years
GROUP BY research_orgs, year
),
retention_rate_with_affiliation AS (
SELECT
retention_rate.research_orgs,
retention_rate.year,
retention_rate.remaining_scientists,
scientist_affiliation.affiliated_scientists,
CASE
  WHEN scientist_affiliation.affiliated_scientists = 0 THEN 0
  ELSE retention_rate.remaining_scientists / scientist_affiliation.affiliated_scientists
END AS retention_rate
FROM retention_rate
JOIN (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS affiliated_scientists
FROM cshdimensionstest.test.simple_sequence_1980_2022
WHERE year >= 1980 AND year <= 2022
GROUP BY research_orgs, year
) AS scientist_affiliation
ON retention_rate.research_orgs = scientist_affiliation.research_orgs
AND retention_rate.year = scientist_affiliation.year)
SELECT
research_orgs,
year,
affiliated_scientists,
remaining_scientists,
retention_rate
FROM retention_rate_with_affiliation;

* One thing that we notice is that the non-migrant percentage is negative.

* The reason why this happens is that we calculate the population number for each year and do not take into account the population that stayed from the previous year. We need to make some adjustments.

* To count only the distinct cumulative population for each year, you could add a column to your query that identifies whether a scientist is still affiliated with the same institution in the next year. Then, you can sum up the number of scientists who are still affiliated with the same institution and divide that by the total number of affiliated scientists to get the cumulative non-migrant percentage.

* non-mobile = what is the percentage of non-mobile researchers who have never left up until that point in time

-- we need to change the `non_migrant_percentage `indicator and align it with the `retention_rate` indicators


In [ ]:
#@title Vis snippet
client = bigquery.Client()

# Make the query
df = pd.io.gbq.read_gbq('''
select *
from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
where node in (
  select node
  from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
  group by node
  having sum(total_flows) > 50000
  order by sum(total_flows) desc)
''', project_id=project_id, dialect='standard')

# Average outgoing
# Create a plotly scatter plot
fig = go.Figure()

for node in df['node'].unique():
    node_df = df[df['node'] == node]
    fig.add_trace(go.Scatter(x=node_df['date_d']
                             , y=node_df['percentage_outflows']
                             , mode='lines+markers'
                             , name=node
                             , marker=dict(color=node_df['total_flows'], showscale=False, colorscale='Blues', opacity=0.5)
                             , hovertemplate='Year: %{x}<br>' + 'Incoming Academic Age: %{y}<br>' + 'Total Flows: %{marker.color:.2f}<br>' + 'Node: ' + node + '<br><extra></extra>'))

# Add a shared color axis
fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Total Flows",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            len=0.5,
            tickangle=-45,
            tickmode='array',
            tickvals=[0, 500, 1000, 5000, 10000, 50000]
        ),
        showscale=True,
    ),
    title="Average Academic Age by Year and Institution",
    xaxis_title="Year",
    yaxis_title="Average Normalized Incoming Academic Age",
    showlegend=True,
)

# Show the plot
pio.show(fig)

# save the plot in plotly graph studio and edit it there
#!pip install chart-studio
#import chart_studio.plotly as py
#py.sign_in(username='Ferreir4', api_key='40YuWFKy73EGkgEjddBA')
# save the plot as an HTML file in your Google Drive folder
#url = py.plot(fig, filename='my_plot', auto_open=False)

# **Data Exports**

#### Countries' Metadata

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.country_coordinates`
"""
countries_metadata = client.query(sql).to_dataframe()
countries_metadata.head(10)

# save the dataset
countries_metadata.to_csv('countries_metadata.csv', index_label='row_no', encoding = 'utf-8-sig')
#files.download('organisations_metadata.csv')

#from google.colab import drive
#drive.mount('/content/drive')
!cp countries_metadata.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Edges

In [ ]:
%%bigquery --project $project_id

select * from cshdimensionstest.test.flows_1980_2022_country_flows limit 100

In [41]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_flows`
ORDER BY geoid_o, geoid_d, date_d
"""
country_edges = client.query(sql).to_dataframe()
country_edges.head(10)

country_edges.to_csv('country_edges.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_edges.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Indicators

In [43]:
%%bigquery --project $project_id
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_production`
where node = 'AT'
ORDER BY node, date_d
limit 50

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate,mean_outgoing_academic_age,...,total_outgoing_pubs,mean_outgoing_pubs,mean_normalized_outgoing_pubs,median_outgoing_pubs,stdev_outgoing_pubs,total_incoming_pubs,mean_incoming_pubs,mean_normalized_incoming_pubs,median_incoming_pubs,stdev_incoming_pubs
0,AT,1980,5,71.4,2,28.6,7,-3,-42.9,1.000,...,161,3.426,0.073,2,2.338196,216,3.323,0.051,2,1.750125
1,AT,1981,11,68.8,5,31.3,16,-6,-37.5,1.545,...,376,4.273,0.049,3,3.190016,293,5.426,0.100,3,4.895404
2,AT,1982,20,46.5,23,53.5,43,3,7.0,2.100,...,559,6.011,0.065,3,5.163616,661,6.178,0.058,3,6.421974
3,AT,1983,19,55.9,15,44.1,34,-4,-11.8,2.263,...,710,8.554,0.103,3,10.936729,827,11.486,0.160,4,16.059832
4,AT,1984,41,53.9,35,46.1,76,-6,-7.9,3.268,...,928,8.286,0.074,4,7.608655,1042,9.304,0.083,3,12.741057
5,AT,1985,44,51.2,42,48.8,86,-2,-2.3,2.568,...,1029,9.894,0.095,4,12.884074,1237,11.349,0.104,4,15.834446
6,AT,1986,51,60.7,33,39.3,84,-18,-21.4,3.431,...,1268,11.221,0.099,4,15.814636,1012,10.327,0.105,4,10.191589
7,AT,1987,114,44.7,141,55.3,255,27,10.6,3.377,...,2862,13.961,0.068,4,19.311944,5696,17.580,0.054,5,26.192506
8,AT,1988,125,62.2,76,37.8,201,-49,-24.4,3.912,...,3947,18.618,0.088,4,29.453906,2142,13.557,0.086,5,13.023920
9,AT,1989,152,52.8,136,47.2,288,-16,-5.6,4.770,...,2800,12.281,0.054,4,13.648094,3522,14.861,0.063,5,18.668360


In [44]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_production`
ORDER BY node, date_d
"""
country_indicators = client.query(sql).to_dataframe()
#country_indicators.head(10)

country_indicators.to_csv('country_indicators.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [45]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_age_groups_country`
ORDER BY node, date_d
"""
country_indicators_age_group_counts = client.query(sql).to_dataframe()
country_indicators_age_group_counts.head(10)

country_indicators_age_group_counts.to_csv('country_indicators_age_group_counts.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_age_group_counts.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [46]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_pub_groups_country`
ORDER BY node, date_d
"""
country_indicators_pub_group_counts = client.query(sql).to_dataframe()
country_indicators_pub_group_counts.head(10)

country_indicators_pub_group_counts.to_csv('country_indicators_pub_group_counts.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_pub_group_counts.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [48]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_sources`
ORDER BY destination_country, date_d, source_rank_1980_2022
"""
country_indicators_top_5_sources = client.query(sql).to_dataframe()
country_indicators_top_5_sources.head(10)

country_indicators_top_5_sources.to_csv('country_indicators_top_K_sources_pairs.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_K_sources_pairs.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [49]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_destinations`
ORDER BY source_country, date_d, destination_rank_1980_2022
"""
country_indicators_top_5_destinations= client.query(sql).to_dataframe()
country_indicators_top_5_destinations.head(10)

country_indicators_top_5_destinations.to_csv('country_indicators_top_K_destinations_pairs.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_K_destinations_pairs.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [50]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_sources_overall`
"""
country_indicators_top_5_sources= client.query(sql).to_dataframe()
country_indicators_top_5_sources.head(10)

country_indicators_top_5_sources.to_csv('country_indicators_top_K_sources_overall.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_K_sources_overall.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [51]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_destinations_overall`
"""
country_indicators_top_5_destinations= client.query(sql).to_dataframe()
country_indicators_top_5_destinations.head(10)

country_indicators_top_5_destinations.to_csv('country_indicators_top_K_destinations_overall.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_K_destinations_overall.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

# PART III - Coverage

1. Validation of trajectories based on ORCID information
2. This is currently work in progress
